In [ ]:
%%html
<link href="http://mathbook.pugetsound.edu/beta/mathbook-content.css" rel="stylesheet" type="text/css" />
<link href="https://aimath.org/mathbook/mathbook-add-on.css" rel="stylesheet" type="text/css" />
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,400italic,600,600italic" rel="stylesheet" type="text/css" />
<link href="https://fonts.googleapis.com/css?family=Inconsolata:400,700&subset=latin,latin-ext" rel="stylesheet" type="text/css" />

**Important:** to view this notebook properly you will need to execute the cell above, which assumes you have an Internet connection.  It should already be selected, or place your cursor anywhere above to select.  Then press the "Run" button in the menu bar above (the right-pointing arrowhead), or press Shift-Enter on your keyboard.

$
\newcommand{\lt}{<}
\newcommand{\gt}{>}
\newcommand{\amp}{&}
$

<div class="mathbook-content"><h1 class="heading"><span class="title">1-5: Two Linear Programs</span></h1><div class="author"><div class="author-name">Robert A. Beezer</div><div class="author-info" /></div></div>

<div class="mathbook-content"><h6 class="heading"><span class="title">Candy Mix</span></h6></div>

<div class="mathbook-content"><p id="p-1">For this very simple example we illustrate how to construct and solve a linear program in Sage.</p></div>

<div class="mathbook-content"><p id="p-2">Make an LP object and create decision variables.</p></div>

In [ ]:
CM = MixedIntegerLinearProgram(maximization=True)
v = CM.new_variable(real=True, nonnegative=True)
x, y = v['x'], v['y']

<div class="mathbook-content"><p id="p-3">With decision variables determined, we can now construct the objective function to maximize: revenue from selling candy.</p></div>

In [ ]:
CM.set_objective(2*x + 2.25*y)

<div class="mathbook-content"><p id="p-4">We have two constraints that keep us from just selling tons and tons of candy.</p></div>

In [ ]:
CM.add_constraint((1/2)*x + (1/3)*y <= 130)
CM.add_constraint((1/2)*x + (2/3)*y <= 170)

<div class="mathbook-content"><p id="p-5">And now we can solve the linear program.  The output is maximum revenue we can achive.</p></div>

In [ ]:
CM.solve()

<div class="mathbook-content"><p id="p-6">Once we know how much revenue we <em class="emphasis">can</em> achieve, we would like to know how to accomplish the feat.</p></div>

In [ ]:
CM.get_values([x, y])

<div class="mathbook-content"><h6 class="heading"><span class="title">OilCo Blending Example</span></h6></div>

<div class="mathbook-content"><p id="p-7">For this more complicated model, we focus more on model formulation, using Sage's tools, which are not quite industrial-strength, but still have the same flavor.  Notice how good variable names and consistency help us avoid little (but potentially serious) mistakes.  We leave in the comments from our scratch worksheet where we first formulated the model.</p></div>

<div class="mathbook-content"><p id="p-8">A maximization LP, with twelve decisions: allocate crude oil to gasoline product.  Totals (of crude and of product) will be useful, so we create seven new variables.  Since they are not strictly necessary, we call them <dfn class="terminology">convenience</dfn> variables.</p></div>

In [ ]:
OCLP = MixedIntegerLinearProgram(maximization=True)
v = OCLP.new_variable(real=True, nonnegative=True)
# 12 decision variables, crude to product
A1R, A1U, A1P = v['A1R'],v['A1U'],v['A1P']
A2R, A2U, A2P = v['A2R'],v['A2U'],v['A2P']
TR, TU, TP = v['TR'],v['TU'],v['TP']
LR, LU, LP = v['LR'],v['LU'],v['LP']
# 7 convenience variables, total crude, total product
A1, A2, T, L, R, U, P = v['A1'], v['A2'], v['T'], v['L'], v['R'], v['U'], v['P']

<div class="mathbook-content"><p id="p-9">The convenience variables are deployed with equality constraints. (Note formatting.)</p></div>

In [ ]:
# set convenience variables
OCLP.add_constraint(A1 == A1R + A1U + A1P)
OCLP.add_constraint(A2 == A2R + A2U + A2P)
OCLP.add_constraint( T ==  TR +  TU +  TP)
OCLP.add_constraint( L ==  LR +  LU +  LP)
OCLP.add_constraint( R == A1R + A2R + TR + LR)
OCLP.add_constraint( U == A1U + A2U + TU + LU)
OCLP.add_constraint( P == A1P + A2P + TP + LP)

<div class="mathbook-content"><p id="p-10">You can begin to appreciate the convenience variables when formulating the objective function.  Profit is revenue less cost.  Here is where dollar amounts are used.  You could “play” with these dollar amounts to see the effect on your production schedule.</p></div>

In [ ]:
# objective is easy with convenience variables
OCLP.set_objective(0.86*R + 0.93*U + 1.06*P - 0.78*A1 - 0.88*A2 - 0.75*T - 1.30*L)

<div class="mathbook-content"><p id="p-11">Now the uncomplicated constraints due to supply of crude, contractual agreements on product, pipeline capacity and technical requirements (unleaded has no lead!).  Storage tanks, trending costs and sale prices, and a multi-day model could prove an interesting expansion of the problem.</p></div>

<div class="mathbook-content"><p id="p-12">Notice the <code class="code-inline tex2jax_ignore">UL</code> variable (lead in unleaded) that we set to zero.  I will argue that this is much more natural (and safer!) than just not having the variable.</p></div>

In [ ]:
# hard limits on production, supply
OCLP.add_constraint(R >=  5000)
OCLP.add_constraint(U >=  5000)
OCLP.add_constraint(P >=  5000)
OCLP.add_constraint(T <= 11000)
OCLP.add_constraint(L <=  6000)
# Alaska pipeline capacity
OCLP.add_constraint(A1 + A2 <= 10000)
# no lead in unleaded
OCLP.add_constraint(LU == 0)

<div class="mathbook-content"><p id="p-13">In blending problems, we assume certain properties blend “linearly”.  What does this mean?  Consider the octane value of our regular gasoline, given that we know the octane values of each crude in the mix.  We take a weighted average (proportions, fractions) of each crude's octane value.  (Note the utility of the convenience variable again.)</p>
\begin{align*}
91(A1R/R) + 97(A2R/R) + 83(TR/R) + 800(LR/R) &\geq 90\\
\left(91(A1R) + 97(A2R) + 83(TR) + 800(LR)\right)/R &\geq 90\\
91(A1R) + 97(A2R) + 83(TR) + 800(LR) &\geq 90R
\end{align*}
<p>In the last step, we have “linearized” the constraint, which is an important technique in model formulation.  Other classes of problems might commonly use similar techniques.</p></div>

In [ ]:
# octane averages
OCLP.add_constraint(91*A1R + 97*A2R + 83*TR + 800*LR >= 90*R)
OCLP.add_constraint(91*A1U + 97*A2U + 83*TU + 800*LU >= 88*U)
OCLP.add_constraint(91*A1P + 97*A2P + 83*TP + 800*LP >= 90*P)

In [ ]:
# sulphur averages, in percent
OCLP.add_constraint(4*A1R + 1*A2R + 2*TR + 0*LR <= 3*R)
OCLP.add_constraint(4*A1U + 1*A2U + 2*TU + 0*LU <= 3*U)
OCLP.add_constraint(4*A1P + 1*A2P + 2*TP + 0*LP <= 3*P)

<div class="mathbook-content"><p id="p-14">Model is complete.  What have we wrought?</p></div>

In [ ]:
OCLP

<div class="mathbook-content"><p id="p-15">Ready, set, solve.</p></div>

In [ ]:
OCLP.solve()

<div class="mathbook-content"><p id="p-16">Certainly senior management is very interested in this daily profit figure.  But there is more to a refinery than that.  Your model is part of all aspects of its operation.</p></div>

<div class="mathbook-content"><p id="p-17">Buyers will want to know how much crude to contract for.</p></div>

In [ ]:
# inputs
OCLP.get_values([A1, A2, T, L])

<div class="mathbook-content"><p id="p-18">Delivery and sales need to know how much product there will be of each type.  Legal will want to know you are meeting your contracts.</p></div>

In [ ]:
# outputs
OCLP.get_values([R, U, P])

<div class="mathbook-content"><p id="p-19">Refinery operations needs to know the right mixes.  Here is the recipe for regular gasoline.</p></div>

In [ ]:
OCLP.get_values([A1R, A2R, TR, LR])